## Web Scraping NBA Games With Python
### Downloading NBA Games Data (Scores, Standings)

In [11]:
# !pip install playwright
# !playwright install

131.1 Mb [                    ] 0% 8.8s131.1 Mb [                    ] 0% 81.5s131.1 Mb [                    ] 0% 66.2s131.1 Mb [                    ] 0% 47.1s131.1 Mb [                    ] 0% 37.2s131.1 Mb [                    ] 0% 31.7s131.1 Mb [                    ] 0% 29.1s131.1 Mb [                    ] 0% 26.7s131.1 Mb [                    ] 0% 25.6s131.1 Mb [                    ] 0% 24.7s131.1 Mb [                    ] 0% 24.1s131.1 Mb [                    ] 0% 23.5s131.1 Mb [                    ] 0% 23.0s131.1 Mb [                    ] 1% 22.5s131.1 Mb [                    ] 1% 22.2s131.1 Mb [                    ] 1% 21.9s131.1 Mb [                    ] 1% 21.7s131.1 Mb [                    ] 1% 21.4s131.1 Mb [                    ] 1% 21.3s131.1 Mb [                    ] 1% 21.1s131.1 Mb [                    ] 1% 20.8s131.1 Mb [                    ] 1% 20.7s131.1 Mb [                    ] 1% 20.6s131.1 Mb [                    ] 1% 20.4s131.1 Mb [                    ] 2% 20.3s1

In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
seasons = list(range(2016, 2023))
seasons

[2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [3]:
DATA_DIR = 'data'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings')
SCORES_DIR = os.path.join(DATA_DIR, 'scores')

In [37]:
async def get_html(url: str, selector: str, sleep:int=5, retries:int=5) -> str:
  html = None
  for i in range(1, retries + 1):
    try:
      async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        # increase sleep time for each retry attempt
        # to avoid being blocked by the server
        page.wait_for_timeout(sleep * i * 1000)
        print(await page.title())
        html = await page.inner_html(selector)
    except PlaywrightTimeout:
      print(f'Timeout error on {url}')
      continue
    else:
      # no retry if html is successfilly retrieved
      break
  return html

In [51]:
async def scrape_season(season: int) -> None:
  url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
  html = await get_html(url, '#content .filter')
  soup = BeautifulSoup(html)
  links = soup.find_all('a')
  href = [link['href'] for link in links]
  standings_pages = [f'https://www.basketball-reference.com{link}' for link in href]
  
  for link in standings_pages:
    save_path = os.path.join(STANDINGS_DIR, link.split('/')[-1])
    if os.path.exists(save_path):
      continue

    html = await get_html(link, '#all_schedule')
    with open(save_path, 'w+') as f:
      f.write(html)

In [53]:
for season in seasons:
  await scrape_season(season)

In [61]:
async def scrape_game(standings_file):
  with open(standings_file, 'r') as f:
    html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    hrefs = [l.get('href') for l in links]
    box_scores = [l for l in hrefs if l and 'boxscore' in l and '.html' in l]
    box_scores = [f'https://www.basketball-reference.com{l}' for l in box_scores]

    for url in box_scores:
      save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
      if os.path.exists(save_path):
        continue

      html = await get_html(url, '#content')
      if not html:
        continue
      with open(save_path, 'w+') as f:
        f.write(html)

In [63]:
standing_files = [os.path.join(STANDINGS_DIR, f) for f in os.listdir(STANDINGS_DIR) if f.endswith('.html')]
for file in standing_files:
  await scrape_game(file)